# Descargando contenido

## # C2_S4 · Manejo de errores en Requests + BeautifulSoup (Demo profesor)
Meta: raise_for_status, timeout+backoff, validar Content-Type/tamaño, arreglar encoding.


## 🎙️ Guion (12–15 min)
0–3’  raise_for_status + política retry/skip/fail  
3–7’  timeout + backoff (429 y 5xx)  
7–10’ validar Content-Type y tamaño  
10–12’ encoding correcto → soup  
12–15’ mini-reto Slack

In [1]:
import requests
from bs4 import BeautifulSoup

import csv
import json

import time, random

# Binario o Texto

diferenciar si el contenido que recibes es texto (string) o binario (bytes).

Cuando haces requests.get(url) puedes revisar:

-> r.text → Devuelve el contenido decodificado como texto (str).

-> r.content → Devuelve el contenido crudo en bytes.

In [2]:
import requests

url = "https://httpbin.org/html"
r = requests.get(url)

print(type(r.text))     # <class 'str'>
print(r.text[:100])     # Primeros 200 caracteres del HTML


<class 'str'>
<!DOCTYPE html>
<html>
  <head>
  </head>
  <body>
      <h1>Herman Melville - Moby-Dick</h1>

     


👉 Aquí r.text es un string porque la respuesta es HTML (texto).
Por eso, si lo quisieras guardar en un archivo, abrirías el archivo con "w" (modo texto):

In [5]:
with open("pagina.html", "w", encoding="utf-8") as f:
    f.write(r.text)

🚀 Ejemplo sencillo: archivo PDF (binario)

In [7]:
import requests

url = "https://www.w3.org/WAI/ER/tests/xhtml/testfiles/resources/pdf/dummy.pdf"
r = requests.get(url)

print(type(r.content))   # <class 'bytes'>
print(r.content[:20])    # Primeros 20 bytes crudos


<class 'bytes'>
b'%PDF-1.4\n%\xc3\xa4\xc3\xbc\xc3\xb6\xc3\x9f\n2'


👉 Aquí r.content es bytes, porque un PDF no es texto.
Entonces necesitas guardar el archivo en modo binario "wb":

el wb significa:

w → write (escritura, abrir el archivo para escribir desde cero, borrando lo anterior).

b → binary (modo binario).


In [8]:
with open("ejemplo.pdf", "wb") as f:
    f.write(r.content)


# Chunks


![ww](cuhnks.png)

🍕 Analogía sencilla:

Un archivo es como una pizza entera 🍕.

Chunk = cada rebanada que te sirven.

Vas guardando rebanada por rebanada en tu plato → al final tienes la pizza completa.

In [9]:
import requests

url = "https://www.w3.org/WAI/ER/tests/xhtml/testfiles/resources/pdf/dummy.pdf"
r = requests.get(url, stream=True, timeout=15)

tamanio_total = 0

with open("ejemplo.pdf", "wb") as f:
    for chunk in r.iter_content(2000):  # cada chunk = 10 bytes (muy pequeñito a propósito)
        if chunk:                     # evita escribir chunks vacíos
            f.write(chunk)
            tamanio_total += len(chunk)
            print(f"Chunk recibido: {len(chunk)} bytes | Total: {tamanio_total} bytes")

print("✅ Archivo descargado en modo 'chunks'")


Chunk recibido: 2000 bytes | Total: 2000 bytes
Chunk recibido: 2000 bytes | Total: 4000 bytes
Chunk recibido: 2000 bytes | Total: 6000 bytes
Chunk recibido: 2000 bytes | Total: 8000 bytes
Chunk recibido: 2000 bytes | Total: 10000 bytes
Chunk recibido: 2000 bytes | Total: 12000 bytes
Chunk recibido: 1264 bytes | Total: 13264 bytes
✅ Archivo descargado en modo 'chunks'


# Nombres de archivo seguros TEMA NO DADO

1) Buscar si el servidor sugiere un nombre
Muchos servidores mandan en la cabecera HTTP:

In [10]:
Content-Disposition: attachment; filename="reporte.pdf"


SyntaxError: illegal target for annotation (2483199984.py, line 1)

Eso significa: el archivo debería llamarse reporte.pdf.

2) Si no hay sugerencia → inventar el nombre

Lo más fácil es usar el último segmento de la URL.


3) Sanitizar (limpiar el nombre)

Reemplazar caracteres inválidos por _.

4) Evitar colisiones (archivos repetidos)

Si ya existe manual.pdf en la carpeta, se debe guardar así:

👨‍💻 Ejemplo Python (simple)

In [13]:
import os
import requests
from urllib.parse import urlparse

def nombre_archivo_seguro(url, response, carpeta="downloads"):

    # 1. Intentar usar Content-Disposition
    cd = response.headers.get("Content-Disposition", "")
    if "filename=" in cd:
        nombre = cd.split("filename=")[-1].strip('" ')
    else:
        # 2. Usar último segmento de la URL
        nombre = os.path.basename(urlparse(url).path) or "descarga"

    # 3. Sanitizar (permitir solo letras, números, ., _, -)
    nombre = "".join(c if c.isalnum() or c in "._-" else "_" for c in nombre)

    # 4. Evitar colisiones
    base, ext = os.path.splitext(nombre)
    contador = 1
    while os.path.exists(os.path.join(carpeta, nombre)):
        nombre = f"{base}({contador}){ext}"
        contador += 1

    return os.path.join(carpeta, nombre)

# =========================
# EJEMPLO DE USO
# =========================
url = "https://www.w3.org/WAI/ER/tests/xhtml/testfiles/resources/pdf/dummy.pdf"
r = requests.get(url, stream=True, timeout=10)

nombre = nombre_archivo_seguro(url, r)
print("Nombre final:", nombre)

# 👇 Aquí GUARDAR el archivo
with open(nombre, "wb") as f:
    for chunk in r.iter_content(4096):
        if chunk:
            f.write(chunk)

print("✅ Archivo descargado:", nombre)


Nombre final: downloads/dummy.pdf
✅ Archivo descargado: downloads/dummy.pdf


# Formatos de guardado


📂 Guardar como CSV

In [24]:
import csv

def guardar_csv(product_list, archivo="resultados/productos.csv"):
    with open(archivo, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=["title", "price", "image_url"])
        writer.writeheader()
        writer.writerows(product_list)

    print(f"✅ Guardado en CSV: {len(product_list)} productos en {archivo}")


📂 Guardar como JSON

In [25]:
import json

def guardar_json(product_list, archivo="resultados/productos.json"):
    with open(archivo, "w", encoding="utf-8") as f:
        json.dump(product_list, f, indent=4, ensure_ascii=False)

    print(f"✅ Guardado en JSON: {len(product_list)} productos en {archivo}")

📂 Guardar como Excel

In [26]:
import pandas as pd

def guardar_excel(product_list, archivo="resultados/productos.xlsx"):
    df = pd.DataFrame(product_list)
    df.to_excel(archivo, index=False)

    print(f"✅ Guardado en Excel: {len(product_list)} productos en {archivo}")


🚀 Ejemplo de uso rápido

In [28]:
# Supongamos que tienes algo como:
product_list = [
    {"title": "Libro A", "price": "10.99", "image_url": "http://.../a.jpg"},
    {"title": "Libro B", "price": "12.50", "image_url": "http://.../b.jpg"},
]

guardar_csv(product_list)
guardar_json(product_list)
guardar_excel(product_list)


✅ Guardado en CSV: 2 productos en resultados/productos.csv
✅ Guardado en JSON: 2 productos en resultados/productos.json
✅ Guardado en Excel: 2 productos en resultados/productos.xlsx
